<a href="https://colab.research.google.com/github/nbacomdados/nbacomdados/blob/main/notebooks/leitura_jogos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
url = 'https://www.espn.com/nba/matchup?gameId=401332963'

In [5]:
dfs = pd.read_html(url)
len(dfs)

4

In [6]:
dfs[0]

,Unnamed: 0,1,2,3,4,OT,T
0,MIL,25,22,35,27,6,115
1,BKN,28,25,28,28,2,111


In [15]:
dfs[1].to_csv('BucksXNets_jogo7_Leste_SemiFinal.csv')

In [13]:
dfs[2]

,TEAM,W,L,PCT,GB,STRK
0,Milwaukee,46,26,0.639,0,L1
1,Indiana,34,38,0.472,12,W1
2,Chicago,31,41,0.431,15,W1
3,Cleveland,22,50,0.306,24,L2
4,Detroit,20,52,0.278,26,L5


In [14]:
dfs[3]

,TEAM,W,L,PCT,GB,STRK
0,Philadelphia,49,23,0.681,0,W2
1,Brooklyn,48,24,0.667,1,W5
2,New York,41,31,0.569,8,W3
3,Boston,36,36,0.500,13,L1
4,Toronto,27,45,0.375,22,L7
